In [2]:
############### Import Libraries ###############
import urllib.request
import zipfile
import os
import pandas as pd
import logging # for logging
import sys
import shutil #to delete the directory contents
import glob
import numpy as np

In [18]:
#path = os.path.join(os.path.dirname(__file__),'downloaded_zips_unzipped')
#filelist = glob.glob('downloaded_zips_unzipped' + "/*.csv")
#read them into pandas
#df_list = [pd.read_csv(file) for file in filelist]
#all_csv_df = pd.concat(df_list)

In [62]:
############### Initializing logging file ###############
root = logging.getLogger()
root.setLevel(logging.DEBUG)

ch1 = logging.FileHandler('problem2_log.log') #output the logs to a file
ch1.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
ch1.setFormatter(formatter)
root.addHandler(ch1)

ch = logging.StreamHandler(sys.stdout ) #print the logs in console as well
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(levelname)s - %(message)s')
ch.setFormatter(formatter)
root.addHandler(ch)


In [63]:
############### Cleanup required directories ###############
if not os.path.exists('downloaded_zips'):
    os.makedirs('downloaded_zips', mode=0o777)
else:
    shutil.rmtree(os.path.join(os.path.dirname(__file__),'downloaded_zips'), ignore_errors=False)
    os.makedirs('downloaded_zips', mode=0o777)

if not os.path.exists('downloaded_zips_unzipped'):
    os.makedirs('downloaded_zips_unzipped', mode=0o777)
else:
    shutil.rmtree(os.path.join(os.path.dirname(__file__), 'downloaded_zips_unzipped'), ignore_errors=False)
    os.makedirs('downloaded_zips_unzipped', mode=0o777)
      
    

NameError: name '__file__' is not defined

In [125]:
def download_zip(url):
    zips = []
    try:
        zips.append(urllib.request.urlretrieve(url, filename= 'downloaded_zips/'+url[-15:]))
        if os.path.getsize('downloaded_zips/'+url[-15:]) <= 4515: #catching empty file
            os.remove('downloaded_zips/'+url[-15:])
            logging.warning('Log file %s is empty. Attempting to download for next date.', url[-15:])
            return False
        else:
            logging.info('Log file %s successfully downloaded', url[-15:])
            return True
    except Exception as e: #Catching file note found
        #print(str(e))
        logging.warning('Log %s not found...Skipping ahead!', url[-15:])
        return True

In [126]:
############### Generate URLs for the inputted year ###############
#http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/2016/Qtr1/log20160101.zip
url_pre = "http://www.sec.gov/dera/data/Public-EDGAR-log-file-data/"
qtr_months = {'Qtr1':['01','02','03'], 'Qtr2':['04','05','06'], 'Qtr3':['07','08','09'], 'Qtr4':['10','11','12']}
year = input("Please enter the year for which you want to analyze the logs:")
valid_years = range(2003,2017)
days = range(1,32)

if not year:
    year = 2003
    logging.warning('Program running for 2003 by default since you did not enter any Year.')

if int(year) not in valid_years:
    logging.error("Invalid year. Please enter a valid year between 2003 and 2016.")
    exit()

url_final = []
for key, val in qtr_months.items():
    for v in val:
        for d in days:
            url = url_pre +str(year) +'/' +str(key) +'/' +'log' +str(year) +str(v) + str(format(d,'02d')) +'.zip'
            if download_zip(url):
                break
            else:
                continue
logging.info('All log files downloaded')


Please enter the year for which you want to analyze the logs:
WARNING - Program running for 2003 by default since you did not enter any Year.
WARNING - Program running for 2003 by default since you did not enter any Year.
WARNING - Log file log20030101.zip is empty. Attempting to download for next date.
WARNING - Log file log20030101.zip is empty. Attempting to download for next date.
INFO - Log file log20030102.zip successfully downloaded
INFO - Log file log20030102.zip successfully downloaded
WARNING - Log file log20030201.zip is empty. Attempting to download for next date.
WARNING - Log file log20030201.zip is empty. Attempting to download for next date.
WARNING - Log file log20030202.zip is empty. Attempting to download for next date.
WARNING - Log file log20030202.zip is empty. Attempting to download for next date.
WARNING - Log file log20030203.zip is empty. Attempting to download for next date.
WARNING - Log file log20030203.zip is empty. Attempting to download for next date.
WA

In [127]:
############### Unzip the logs and extract csv ###############
try:
    zip_files = os.listdir('downloaded_zips')
    for f in zip_files:
        z = zipfile.ZipFile(os.path.join('downloaded_zips', f), 'r')
        for file in z.namelist():
            if file.endswith('.csv'):
                z.extract(file, r'downloaded_zips_unzipped')
    logging.info('Zip files successfully extracted to folder: downloaded_zips_unzipped.')
except Exception as e:
        logging.error(str(e))
        exit()

INFO - Zip files successfully extracted to folder: downloaded_zips_unzipped.
INFO - Zip files successfully extracted to folder: downloaded_zips_unzipped.


In [3]:
############### Load the csvs into dataframe ###############
filelists = glob.glob('downloaded_zips_unzipped' + "/*.csv")
#url = '{}'
all_csv_df_dict = {period: pd.read_csv(period) for period in filelists}


In [135]:
df = all_csv_df_dict['downloaded_zips_unzipped\\log20030301.csv']

In [6]:
############### HANDLING MISSING VALUES for one dataframe at a time ###############
for key, val in all_csv_df_dict.items():
    df = all_csv_df_dict[key]
    #remove rows which have no ip, cik or accession
    df.dropna(subset=['cik'])
    df.dropna(subset=['accession'])
    df.dropna(subset=['ip'])
    df.dropna(subset=['date'])
    df.dropna(subset=['time'])
    #replace nan with the most used browser in data.
    max_browser = pd.DataFrame(df.groupby('browser').size().rename('cnt')).idxmax()[0]
    df['browser'] = df['browser'].fillna(max_browser)
    # replace nan idx with max idx
    max_idx = pd.DataFrame(df.groupby('idx').size().rename('cnt')).idxmax()[0]
    df['idx'] = df['idx'].fillna(max_idx)
    # replace nan code with max code
    max_code = pd.DataFrame(df.groupby('code').size().rename('cnt')).idxmax()[0]
    df['code'] = df['code'].fillna(max_code)
    # replace nan norefer with zero
    df['norefer'] = df['norefer'].fillna('0')
    # replace nan noagent with zero
    df['noagent'] = df['noagent'].fillna('0')
    # replace nan find with max find
    max_find = pd.DataFrame(df.groupby('find').size().rename('cnt')).idxmax()[0]
    df['find'] = df['find'].fillna(max_find)
    # replace nan crawler with zero
    df['crawler'] = df['crawler'].fillna('0')
    # replace nan extention with max extention
    max_extention = pd.DataFrame(df.groupby('extention').size().rename('cnt')).idxmax()[0]
    df['extention'] = df['extention'].fillna(max_extention)
    # replace nan extention with max extention
    max_zone = pd.DataFrame(df.groupby('zone').size().rename('cnt')).idxmax()[0]
    df['zone'] = df['zone'].fillna(max_zone)
    # find mean of the size and replace null values with the mean
    size_mean = df['size'].mean(axis=0) #mean of size
    df['size'] = df['size'].fillna(size_mean)
    df['idx'] = df['idx'].fillna(df['idx'].max(axis=0))
    
    #Compute mean size
    df['size_mean'] = df['size'].mean(axis=0)
    #Compute maximum used browser
    df['max_browser'] = pd.DataFrame(df.groupby('browser').size().rename('cnt')).idxmax()[0]


In [7]:
############### Combining dataframe and write to a csv ###############
master_df = pd.concat(all_csv_df_dict)
master_df.to_csv('master_csv.csv')
# write csv for summary of combined data.
master_df_summary = master_df.describe()
master_df_summary.to_csv('master_df_summary.csv')


In [163]:
############### HANDLING MISSING VALUES ############### OLD
#replace nan with the most used browser in data.
max_browser = pd.DataFrame(df.groupby('browser').size().rename('cnt')).idxmax()[0]
df['browser'] = df['browser'].fillna(max_browser)

# replace nan idx with max idx
max_idx = pd.DataFrame(df.groupby('idx').size().rename('cnt')).idxmax()[0]
df['idx'] = df['idx'].fillna(max_idx)

# replace nan code with max code
max_code = pd.DataFrame(df.groupby('code').size().rename('cnt')).idxmax()[0]
df['code'] = df['code'].fillna(max_code)

# replace nan norefer with zero
df['norefer'] = df['norefer'].fillna('0')

# replace nan noagent with zero
df['noagent'] = df['noagent'].fillna('0')

# replace nan find with max find
max_find = pd.DataFrame(df.groupby('find').size().rename('cnt')).idxmax()[0]
df['find'] = df['find'].fillna(max_find)

# replace nan crawler with zero
df['crawler'] = df['crawler'].fillna('0')

# replace nan extention with max extention
max_extention = pd.DataFrame(df.groupby('extention').size().rename('cnt')).idxmax()[0]
df['extention'] = df['extention'].fillna(max_extention)

# replace nan extention with max extention
max_zone = pd.DataFrame(df.groupby('zone').size().rename('cnt')).idxmax()[0]
df['zone'] = df['zone'].fillna(max_zone)


# find mean of the size and replace null values with the mean
size_mean = df['size'].mean(axis=0) #mean of size
df['size'] = df['size'].fillna(size_mean)
df['idx'] = df['idx'].fillna(df['idx'].max(axis=0))

#remove rows which have no ip, cik or accession
df.dropna(subset=['cik'])
df.dropna(subset=['accession'])
df.dropna(subset=['ip'])
df.dropna(subset=['date'])
df.dropna(subset=['time'])


In [ ]:
############### SUMMARY METRICS ###############
# Calculating Summary metrics
df.describe() # output this

#Compute mean size
df['size_mean'] = df['size'].mean(axis=0)

#Compute maximum used browser
df['max_browser'] = pd.DataFrame(df.groupby('browser').size().rename('cnt')).idxmax()[0]